In [1]:
import tqdm
import mlmi
import mlmi3
import numpy as np
import scipy.stats
import pandas as pd
import sklearn
import sklearn.covariance
import networkx as nx
import matplotlib.pyplot as plt
% matplotlib inline
import seaborn as sns
sns.set_style('ticks')
palette = sns.color_palette("RdBu_r", n_colors=25)

In [2]:
import os
if not os.path.exists('water-treatment.csv'):
    !wget -O water-treatment.csv https://archive.ics.uci.edu/ml/machine-learning-databases/water-treatment/water-treatment.data

In [3]:
df_raw = pd.read_csv('water-treatment.csv', header=None, na_values='?')
df = df_raw.drop(df_raw.columns[0], axis=1).dropna()
df = (df - df.mean()) / df.std()
print('Number of records: ', len(df_raw), '->', len(df))
df[:5]

Number of records:  527 -> 380


,1,2,3,4,5,6,7,8,9,10,...,29,30,31,32,33,34,35,36,37,38
3,-0.342789,0.525093,0.315869,0.250450,1.561485,-0.294765,0.374517,-0.063891,2.442977,-0.227956,...,0.932863,-0.403717,0.409139,0.521996,0.460589,0.347041,0.123252,0.481069,-0.002119,0.207640
8,-1.199077,0.095652,-0.529415,0.266743,0.399040,-0.277827,0.650968,-0.063891,-0.557492,-0.674468,...,-0.373482,0.467183,-1.180861,0.207854,0.215397,-0.993873,0.490159,-0.551941,0.096486,0.062419
9,0.273558,-0.119068,-0.106773,-0.287239,0.865715,-0.227013,0.643069,0.108054,1.007529,-0.227956,...,0.349952,-1.200925,0.061808,0.017124,0.619242,-0.242581,-0.096892,-1.135287,-0.511577,0.110826
10,0.732863,-0.677341,0.315869,-0.010248,0.628135,0.027056,0.485097,0.280000,-0.148452,1.111580,...,0.243259,-2.580966,-1.011055,0.151757,-10.457656,-4.084632,-12.828564,-5.145796,-7.660418,-15.185756
14,0.518317,0.525093,-0.952057,-0.710873,-0.636129,-0.328641,-0.273166,-0.751675,-0.427920,-1.120981,...,0.165190,-0.430514,-1.975861,-0.072631,-0.217295,-0.699062,-0.463799,-0.843614,-1.875609,0.014013


In [4]:
df = df.loc[:, :10]
df[:5]

,1,2,3,4,5,6,7,8,9,10
3,-0.342789,0.525093,0.315869,0.250450,1.561485,-0.294765,0.374517,-0.063891,2.442977,-0.227956
8,-1.199077,0.095652,-0.529415,0.266743,0.399040,-0.277827,0.650968,-0.063891,-0.557492,-0.674468
9,0.273558,-0.119068,-0.106773,-0.287239,0.865715,-0.227013,0.643069,0.108054,1.007529,-0.227956
10,0.732863,-0.677341,0.315869,-0.010248,0.628135,0.027056,0.485097,0.280000,-0.148452,1.111580
14,0.518317,0.525093,-0.952057,-0.710873,-0.636129,-0.328641,-0.273166,-0.751675,-0.427920,-1.120981


In [5]:
d = len(df.columns)
cmi = np.zeros([d, d])
X = df.values
for i, j in tqdm.tqdm([(i, j) for i in range(d) for j in range(d) if i < j]):
    mask = (np.arange(d) != i) & (np.arange(d) != j)
    mi_xz = mlmi.mutual_information(X[:, i].reshape(-1, 1), X[:, mask], sigma=0.5, n_b=200, maxiter=100000)
    mi_yz = mlmi.mutual_information(X[:, j].reshape(-1, 1), X[:, mask], sigma=0.5, n_b=200, maxiter=100000)
    mi_xyz = mlmi3.mutual_information(X[:, i].reshape(-1, 1), X[:, j].reshape(-1, 1), X[:, mask], sigma=0.5, n_b=200, maxiter=100000)
    cmi[j, i] = cmi[i, j] = mi_xyz - (mi_xz + mi_yz)

cmi

 89%|████████▉ | 40/45 [16:05<01:38, 19.76s/it]

Exception: Optimization failed.

In [ ]:
model = sklearn.covariance.GraphLasso(alpha=0.3, verbose=True)
model.fit(X)

In [ ]:
f, axes = plt.subplots(1, 2, figsize=(16, 6))
axes[0].set_title('precision')
axes[1].set_title('cmi > 0.05')
sns.heatmap(np.abs(model.precision_), ax=axes[0], annot=True, center=0, cmap=palette)
sns.heatmap(cmi * (cmi > 0.05), ax=axes[1], annot=True, center=0, cmap=palette)
plt.savefig('cmi-wtp.png', dpi=120)

In [ ]:
f, axes = plt.subplots(2, 2, figsize=(10, 7))
cnt = 0
axes[0, 0].set_title('only cmi: 2-6')
axes[0, 1].set_title('only cmi: 3-6')
axes[1, 0].set_title('only glasso: 5-7')
axes[1, 1].set_title('only glasso: 0-4')
for i, j in [(2, 6), (3, 6), (5, 7), (0, 4)]:
    axes[int(cnt/2)][int(cnt%2)].scatter(X[:, i], X[:, j])
    cnt += 1